In [ ]:
import math
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

%matplotlib inline

In [ ]:
water_atom_type=2
#water_atom_type=5

In [ ]:
#read all parameters of header
time_counter=0
counter=0
timeSteps=[]
boxDim=np.zeros((3,2)) #2 is for low and high dimensions and 3 is for x, y, z
num_frame=0


with open(r'C:\vmd\nvt_flow') as fp:
#with open('/content/drive/My Drive/Case/nanoBloodProject/MSD/disc/12rc/170lig/bare/rc1.0/nvt') as fp:

        for line in fp:
            
            if 'ITEM: TIMESTEP' in line:  
                
                for line in fp: # now you are at the lines you want
                    if 'ITEM: NUMBER OF ATOMS' in line: 
                     break 
                   
                    line=line.rstrip()# to remove any white spaces at the end of the string                  
                    #
                    timeSteps.append(line)# add the string in to the list
                    num_frame+=1
                    

            if 'ITEM: NUMBER OF ATOMS' in line:                                         
                for line in fp: # now you are at the lines you want
                    if 'ITEM: BOX BOUNDS' in line: 
                     break 
                   
                    line=line.rstrip()# to remove any white spaces at the end of the string                  
                    #
                    num_atom=int(line)  

            if 'ITEM: BOX BOUNDS' in line: 
                time_counter+=1
                for line in fp: # now you are at the lines you want
                    if 'ITEM: ATOMS' in line: 
                     break 
                    
                     line=line.rstrip()# to remove any white spaces at the end of the string                  
                    
                    
                    
                    if time_counter==1: #thankGOD! to put box dimension in a matrix
                        line_split = line.split()
                                                    
                        boxDim[counter,0]=line_split[0]
                        
                        boxDim[counter,1]=line_split[1]
                        counter+=1
                  

In [ ]:
x_low=boxDim[0,0]
y_low=boxDim[1,0]
z_low=boxDim[2,0]

x_high=boxDim[0,1]
y_high=boxDim[1,1]
z_high=boxDim[2,1]

Lx=x_high-x_low
Ly=y_high-y_low
Lz=z_high-z_low
print (Lz)

In [ ]:
#read coordinate part
time_counter=0
lines=[]


with open(r'C:\vmd\nvt_flow') as fp:
#with open('/content/drive/My Drive/Case/nanoBloodProject/MSD/rod/8rc/76lig/homo/corNpAij/nvt') as fp:  
       
       for line in fp:
            atom_counter=0
            
            if 'ITEM: ATOMS' in line:
                time_counter+=1                                         
                
                for line in fp: # now you are at the lines you want
                    atom_counter+=1

                    #x(time_counter,atom_counter)=df.loc[line,'x'] 
                                        
                    if 'ITEM: TIMESTEP' in line: 
                     break 
                    
                    
                    line=line.rstrip()# to remove any white spaces at the end of the string                  
                    lines.append(line)# add the string in to the list
                    #lines[time_counter][atom_counter]=line# add the string in to the list

In [ ]:
#remove empty lines from the list
while("" in lines) : 
    lines.remove("") 

In [ ]:
#data frame for the whole dump file without separating by frame
df=pd.DataFrame(lines, columns=['atomID']) # to make a data frame of pandas from the list and put atomID name on the column
df[['atomID','atomType','x','y','z','Vx']] = df.atomID.str.split(expand=True,) # to split one column to multiple columns and allocate name for each column
#df

In [ ]:
num_atom
num_frame

In [ ]:
#we want velocity profile of water so we need to just read velocity of atom type 3 which is water. thankGOD!
type_frame=np.zeros((num_frame,num_atom))
VX=np.zeros((num_frame,num_atom))
Z=np.zeros((num_frame,num_atom))

for i in range (0, num_frame):
   type_frame[i,:]=df.atomType.values[(i*num_atom):((i*num_atom)+num_atom)] #thankGOD! to read x values at each frame and put in an array
   VX[i,:]=df.Vx.values[(i*num_atom):((i*num_atom)+num_atom)]
   Z[i,:]=df.z.values[(i*num_atom):((i*num_atom)+num_atom)]
   #print(i, VX[i,:])

In [ ]:
for i in range (0, num_frame):
  num_atom_type=0
  for k in range (0, num_atom):
     if type_frame[i,k]==water_atom_type: #we only read atoms of water with atom type 3
       num_atom_type+=1

In [ ]:
print (num_atom_type)#number of water beads

very important: to read all lines in the entire dump file( all the lines of all the frames)

In [ ]:
Vx_frame=np.zeros ((num_frame+1,num_atom+1))#velocity of each water bead at each frame. 
z_frame=np.zeros ((num_frame+1,num_atom+1)) #z dimension of each water bead at each frame


In [ ]:

for i in range (0, num_frame):
  

  
  for k in range (0, num_atom):
     if type_frame[i,k]==water_atom_type:
       #atom_counter+=1
       
        
       #Vx_frame[time_counter,k]=df.Vx.values[k]        
       #z_frame[time_counter,k]=df.z.values[k]
       
       Vx_frame[i,k]=VX[i,k]       
       z_frame[i,k]=Z[i,k] 
    
       #Vx_frame[i,atom_counter]=VX[i,atom_counter]        
       #z_frame[i,atom_counter]=Z[i,atom_counter] 
       #print (i,k,Vx_frame[i,k],z_frame[i,k])
       #df = pd.DataFrame(z_frame) #to have velocity of each frame
       #df.T.to_excel(r'C:\vmd\z_frame.xlsx')

use the density profile method for nex step

In [ ]:
layer_thickness=0.25
#layer_thickness=1.0
num_bin=int(Lz/layer_thickness)
print(Lz,num_bin)

In [ ]:
z_dim_layer=np.zeros ((num_frame+1,num_bin+1))


for i in range (0, num_frame):
    for k in range (1, num_bin+1):
        
        z_dim_layer[i][k]=z_dim_layer[i][k-1]+layer_thickness
        #print(k,z_dim_layer[i][k-1],z_dim_layer[i][k])

In [ ]:
#ThankGOD! find the velocity of all beads in each bin (layer of z dimension)for all frames.thankGOD!thankGOD!

Vx_bin_frame=np.zeros ((num_frame+1,num_bin+1))#velocity of each water bead at each layer of the z axis(bin) at each frame

Vx_bin_ave_frame=np.zeros ((num_frame+1,num_bin+1))

atom_bin_counter_frame=np.zeros ((num_frame+1,num_bin+1))


for i in range (0, num_frame):
#for i in range (0, 2): 

 
 for j in range (0, num_atom):
    
    
    
    for k in range (1, num_bin+1):
        
        
        #if (z_frame[time_counter][atom_counter] > z_dim_layer[time_counter][bin_counter-1]) and (z_frame[time_counter][atom_counter] <= z_dim_layer[time_counter][bin_counter]):
        if (z_frame[i][j] > z_dim_layer[i][k-1]) and (z_frame[i][j] < z_dim_layer[i][k]):
               
            
             atom_bin_counter_frame[i][k]+=1
             #print(i,k,time_counter,bin_counter,atom_bin_counter_frame[time_counter][bin_counter])          
             
             Vx_bin_frame[i][k]+= Vx_frame[i][j]#we fill aray with time_counter and bin_counter instead of i and K bc we go over i and k but may be in all i and k we donot have this conditon to have Vx so we just fill the array by counting when we have the required condition thankGOD! 
             #print(i,k,atom_bin_counter_frame[i][k],z_frame[i][j], z_dim_layer[i][k-1], z_dim_layer[i][k]) 
            
             

# for all frames (non time ave)

In [ ]:
#velocity at each frame or ave of velovcity of several bins at each frame (non time ave velocity profile)


for i in range (0, num_frame):
  
  
  for k in range (1, num_bin+1):
       
    if atom_bin_counter_frame[i][k] == 0:
       #Vx_bin_ave_frame[i][k] = 0
       atom_bin_counter_frame[i][k]=1 #to prevent deviding Vx_bin_frame[i][k] by zero and get an nan value. thankGOD!
        
    #ave of velocity of all beads in each bin of each frame. so for one frame it shows velocity of all beads in all bins and we use it to show velocity profile of each frame
    Vx_bin_ave_frame[i][k]=Vx_bin_frame[i][k]/atom_bin_counter_frame[i][k]
       
    
        
    print(i,k, atom_bin_counter_frame[i][k],Vx_bin_frame[i][k], Vx_bin_ave_frame[i][k]) 


In [ ]:
df = pd.DataFrame(Vx_bin_ave_frame) #to have velocity of each frame
df.T.to_excel(r'C:\vmd\velocity_profile0.25.xlsx')

#df = pd.DataFrame(Vx_overall_bins_ave)#to have ave of velocity of all frames 
#df.to_excel(r'C:\vmd\velocity_profile2.xlsx')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\velocity_profile.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

ax=Sheet1.plot(kind='line', y= 0 , color='gray', linewidth=0.5, label='first')

#Sheet1.plot(kind='line', y= 1 , color='blue', linewidth=0.5, label='second', ax=ax)
#ax.legend()
#Sheet1.plot(kind='line',  y= 2, color='red', linewidth=0.5, label='third', ax=ax)
#Sheet1.plot(kind='line',  y= 3, color='green', linewidth=0.5, label='forth', ax=ax)
#Sheet1.plot(kind='line',  y= 4, color='orange', linewidth=0.5, label='forth', ax=ax)
Sheet1.plot(kind='line',  y= 5, color='black', linewidth=0.5, label='fifth', ax=ax)
#Sheet1.plot(kind='line',  y= 6, color='brown', linewidth=0.5, label='sixth', ax=ax)
#Sheet1.plot(kind='line',  y= 7, color='yellow', linewidth=0.5, label='seventh', ax=ax)
#Sheet1.plot(kind='line',  y= 8, color='cyan', linewidth=0.5, label='eigth', ax=ax)
#Sheet1.plot(kind='line',  y= 9, color='pink', linewidth=0.5, label='ninth', ax=ax)
#Sheet1.plot(kind='line',  y= 10, color='purple', linewidth=0.5, label='tenth', ax=ax)

ax.legend()
ax.set_xlabel('bins ()')
ax.set_ylabel('velocity ()') 
pylab.xlim([0,20,1])
pylab.ylim([-0.0001,0.0013])# define the range of y coordinate
#plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
pylab.axhline(0, color='black',linewidth=0.5)#line at y=0
plt.savefig(r'C:\vmd\velocity_profile.png')

# time ave(each frame velocity subtract from first frame)

In [ ]:
# difference of Vx of all frames from the first frame
d_Vx=np.zeros ((num_frame+1,num_bin+1))

for i in range (0, 1):#thank GOD! 
    
 for t in range (1, num_frame):
        
    for k in range (1, num_bin+1):
       
        d_Vx[t][k] = Vx_bin_ave_frame[t][k] - Vx_bin_ave_frame[i][k] 
        
        print(i,t,k,d_Vx[t][k])

In [ ]:
#ave of velovcity of several frames. thankGOD!
Vx_ave=np.zeros ((num_frame+1,num_bin+1))

frame_counter=0
for i in range (0, num_frame):
    frame_counter+=1
    
    for k in range (1, num_bin+1):
        #thankGOD!  
            
       Vx_ave[frame_counter][k]= d_Vx[frame_counter][k]/frame_counter
       print(i,frame_counter, d_Vx[frame_counter][k],Vx_ave[frame_counter][k]) 

In [ ]:
#df = pd.DataFrame(Vx_bin_ave_frame) #to have velocity of each frame
#df.T.to_excel(r'C:\vmd\velocity_profile.xlsx')

df = pd.DataFrame(Vx_ave)#to have ave of velocity of all frames 
df.T.to_excel(r'C:\vmd\velocity_profile_ave.xlsx')
    

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\velocity_profile_ave.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

#ax=Sheet1.plot(kind='line', y= 0 , color='blue', linewidth=0.5, label='ave')

ax=Sheet1.plot(kind='line', y= 1 , color='blue', linewidth=0.5, label='first')
#ax.legend()
Sheet1.plot(kind='line',  y= 2, color='red', linewidth=0.5, label='second', ax=ax)
Sheet1.plot(kind='line',  y= 3, color='green', linewidth=0.5, label='third', ax=ax)
Sheet1.plot(kind='line',  y= 4, color='orange', linewidth=0.5, label='forth', ax=ax)
Sheet1.plot(kind='line',  y= 5, color='black', linewidth=0.5, label='fifth', ax=ax)
Sheet1.plot(kind='line',  y= 6, color='brown', linewidth=0.5, label='sixth', ax=ax)
ax.legend()
ax.set_xlabel('bins ()')
ax.set_ylabel('velocity ()') 
pylab.xlim([0,120])
pylab.ylim([-0.0001,0.00025])# define the range of y coordinate
#plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
pylab.axhline(0, color='black',linewidth=0.5)#line at y=0
plt.savefig(r'C:\vmd\velocity_profile7.png')

# average of several frames

In [ ]:
#ave window 3
#overall velocity of beads in all frames. we use frame_counter bc so we can not devide by i which is 0 at the beginning

v_overall_bins=np.zeros ((num_bin+1)) #is the totall for all frames
v_overall_bins_frames=np.zeros ((num_frame+1,num_bin+1))

frame_counter=0
for i in range (0, num_frame):
    frame_counter+=1
    j=frame_counter-3  #ave window 3

    for k in range (1, num_bin+1):
        
       
    
       if j < 1:
           v_overall_bins[k] += Vx_bin_ave_frame[frame_counter][k]
           
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/frame_counter #overall velocity of beads in all frames
           #print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])
           
       else:
           v_overall_bins[k] = Vx_bin_ave_frame[j+1][k]+Vx_bin_ave_frame[j+2][k]+Vx_bin_ave_frame[j+3][k] #it is like time/ave window 3 of lammps 
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/3
    
    
        #thankGOD!  
            
       
       
       print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])

In [ ]:
#ave window 5

v_overall_bins=np.zeros ((num_bin+1)) #is the totall for all frames
v_overall_bins_frames=np.zeros ((num_frame+1,num_bin+1))

frame_counter=0
for i in range (0, num_frame):
    frame_counter+=1
    j=frame_counter-5  #ave window 5

    for k in range (1, num_bin+1):
        
       
    
       if j < 1:
           v_overall_bins[k] += Vx_bin_ave_frame[frame_counter][k]
           
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/frame_counter #overall velocity of beads in all frames
           #print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])
           
       else:
           v_overall_bins[k] = Vx_bin_ave_frame[j+1][k]+Vx_bin_ave_frame[j+2][k]+Vx_bin_ave_frame[j+3][k]+Vx_bin_ave_frame[j+4][k]+Vx_bin_ave_frame[j+5][k] #it is like time/ave window 3 of lammps 
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/5
    
    
        #thankGOD!  
            
       
       
       print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])

In [ ]:
#ave window 7

v_overall_bins=np.zeros ((num_bin+1)) #is the totall for all frames
v_overall_bins_frames=np.zeros ((num_frame+1,num_bin+1))

frame_counter=0
for i in range (0, num_frame):
    frame_counter+=1
    j=frame_counter-7  #ave window 7

    for k in range (1, num_bin+1):
        
       
    
       if j < 1:
           v_overall_bins[k] += Vx_bin_ave_frame[frame_counter][k]
           
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/frame_counter #overall velocity of beads in all frames
           #print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])
           
       else:
           v_overall_bins[k] = Vx_bin_ave_frame[j+1][k]+Vx_bin_ave_frame[j+2][k]+Vx_bin_ave_frame[j+3][k]+Vx_bin_ave_frame[j+4][k]+Vx_bin_ave_frame[j+5][k]+Vx_bin_ave_frame[j+6][k]+Vx_bin_ave_frame[j+7][k] 
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/7
    
    
        #thankGOD!  
            
       
       
       print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])

In [ ]:
#ave window 10

v_overall_bins=np.zeros ((num_bin+1)) #is the totall for all frames
v_overall_bins_frames=np.zeros ((num_frame+1,num_bin+1))

frame_counter=0
for i in range (0, num_frame):
    frame_counter+=1
    j=frame_counter-10  #ave window 10

    for k in range (1, num_bin+1):
        
       
    
       if j < 1:
           v_overall_bins[k] += Vx_bin_ave_frame[frame_counter][k]
           
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/frame_counter #overall velocity of beads in all frames
           #print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])
           
       else:
           v_overall_bins[k] = Vx_bin_ave_frame[j+1][k]+Vx_bin_ave_frame[j+2][k]+Vx_bin_ave_frame[j+3][k]+Vx_bin_ave_frame[j+4][k]+Vx_bin_ave_frame[j+5][k]+Vx_bin_ave_frame[j+6][k]+Vx_bin_ave_frame[j+7][k]+Vx_bin_ave_frame[j+8][k]+Vx_bin_ave_frame[j+9][k]+Vx_bin_ave_frame[j+10][k] 
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/10
    
    
        #thankGOD!  
            
       
       
       print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])

In [ ]:
#df = pd.DataFrame(Vx_bin_ave_frame) #to have velocity of each frame
#df.T.to_excel(r'C:\vmd\velocity_profile.xlsx')

df = pd.DataFrame(v_overall_bins_frames)#to have ave of velocity of all frames 
df.T.to_excel(r'C:\vmd\velocity_profile_ave_frames.xlsx')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\velocity_profile_ave_frames.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

#ax=Sheet1.plot(kind='line', y= 0 , color='blue', linewidth=0.5, label='ave')

ax=Sheet1.plot(kind='line', y= 1 , color='blue', linewidth=0.5, label='first')
#ax.legend()
Sheet1.plot(kind='line',  y= 2, color='red', linewidth=0.5, label='second', ax=ax)
Sheet1.plot(kind='line',  y= 3, color='green', linewidth=0.5, label='third', ax=ax)
Sheet1.plot(kind='line',  y= 4, color='orange', linewidth=0.5, label='forth', ax=ax)
Sheet1.plot(kind='line',  y= 5, color='black', linewidth=0.5, label='fifth', ax=ax)
Sheet1.plot(kind='line',  y= 6, color='brown', linewidth=0.5, label='sixth', ax=ax)
Sheet1.plot(kind='line',  y= 7, color='yellow', linewidth=0.5, label='seventh', ax=ax)
Sheet1.plot(kind='line',  y= 8, color='cyan', linewidth=0.5, label='eigth', ax=ax)
Sheet1.plot(kind='line',  y= 9, color='pink', linewidth=0.5, label='ninth', ax=ax)
Sheet1.plot(kind='line',  y= 10, color='purple', linewidth=0.5, label='tenth', ax=ax)
ax.legend()
ax.set_xlabel('bins ()')
ax.set_ylabel('velocity ()') 
pylab.xlim([0,120])
pylab.ylim([-0.0001,0.0013])# define the range of y coordinate
#plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
pylab.axhline(0, color='black',linewidth=0.5)#line at y=0
plt.savefig(r'C:\vmd\velocity_profile_ave_frames7.png')

# ave all frames

In [ ]:
#ave window 7

v_overall_bins=np.zeros ((num_bin+1)) #is the totall for all frames
v_overall_bins_frames=np.zeros ((num_frame+1,num_bin+1))

frame_counter=0
for i in range (0, num_frame):
    frame_counter+=1
    j=frame_counter-7  #ave window 7

    for k in range (1, num_bin+1):
        
       
    
       #if j < 1:
           v_overall_bins[k] += Vx_bin_ave_frame[frame_counter][k]
           
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/frame_counter #overall velocity of beads in all frames
           #v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/1
        #print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])
           
       #else:
           #v_overall_bins[k] = Vx_bin_ave_frame[j+1][k]+Vx_bin_ave_frame[j+2][k]+Vx_bin_ave_frame[j+3][k]+Vx_bin_ave_frame[j+4][k]+Vx_bin_ave_frame[j+5][k]+Vx_bin_ave_frame[j+6][k]+Vx_bin_ave_frame[j+7][k] 
           #v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/7
    
    
        #thankGOD!  
            
       
       
           print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])

In [ ]:
#ave window 10

v_overall_bins=np.zeros ((num_bin+1)) #is the totall for all frames
v_overall_bins_frames=np.zeros ((num_frame+1,num_bin+1))

frame_counter=0
for i in range (0, num_frame):
    frame_counter+=1
    j=frame_counter-10  #ave window 10

    for k in range (1, num_bin+1):
        
       
    
       #if j < 1:
           v_overall_bins[k] += Vx_bin_ave_frame[frame_counter][k]
           
           v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/frame_counter #overall velocity of beads in all frames
           #v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/1
        #print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])
           
       #else:
           #v_overall_bins[k] = Vx_bin_ave_frame[j+1][k]+Vx_bin_ave_frame[j+2][k]+Vx_bin_ave_frame[j+3][k]+Vx_bin_ave_frame[j+4][k]+Vx_bin_ave_frame[j+5][k]+Vx_bin_ave_frame[j+6][k]+Vx_bin_ave_frame[j+7][k] 
           #v_overall_bins_frames[frame_counter][k] = v_overall_bins[k]/7
    
    
        #thankGOD!  
            
       
       
           print(i,k,frame_counter, v_overall_bins[k], v_overall_bins_frames[frame_counter][k])

In [ ]:
#df = pd.DataFrame(Vx_bin_ave_frame) #to have velocity of each frame
#df.T.to_excel(r'C:\vmd\velocity_profile.xlsx')

df = pd.DataFrame(v_overall_bins_frames)#to have ave of velocity of all frames 
df.T.to_excel(r'C:\vmd\velocity_profile_ave_frames_all.xlsx')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pylab 

df =pd.ExcelFile(r'C:\vmd\velocity_profile_ave_frames_all.xlsx')# the name of execl file should start with capital word. Rdf is correct but not rdf 
Sheet1 =pd.read_excel(df,header=0)

#ax=Sheet1.plot(kind='line', y= 0 , color='blue', linewidth=0.5, label='ave')

ax=Sheet1.plot(kind='line', y= 1 , color='blue', linewidth=0.5, label='first')
#ax.legend()
Sheet1.plot(kind='line',  y= 2, color='red', linewidth=0.5, label='second', ax=ax)
Sheet1.plot(kind='line',  y= 3, color='green', linewidth=0.5, label='third', ax=ax)
Sheet1.plot(kind='line',  y= 4, color='orange', linewidth=0.5, label='forth', ax=ax)
Sheet1.plot(kind='line',  y= 5, color='black', linewidth=0.5, label='fifth', ax=ax)
Sheet1.plot(kind='line',  y= 6, color='brown', linewidth=0.5, label='sixth', ax=ax)
Sheet1.plot(kind='line',  y= 7, color='yellow', linewidth=0.5, label='seventh', ax=ax)
Sheet1.plot(kind='line',  y= 8, color='cyan', linewidth=0.5, label='eigth', ax=ax)
Sheet1.plot(kind='line',  y= 9, color='pink', linewidth=0.5, label='ninth', ax=ax)
Sheet1.plot(kind='line',  y= 10, color='purple', linewidth=0.5, label='tenth', ax=ax)
ax.legend()
ax.set_xlabel('bins ()')
ax.set_ylabel('velocity ()') 
pylab.xlim([0,120])
pylab.ylim([-0.0001,0.0013])# define the range of y coordinate
#plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))#show the x and y axis in scientific mode
pylab.axhline(0, color='black',linewidth=0.5)#line at y=0
plt.savefig(r'C:\vmd\velocity_profile_ave_frames_all.png')